In [1]:
!pip install face_recognition opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=0aaab730d2256d60267151c99941ac2e6d8ead27d7d48cb49281983354ca03c9
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [2]:
import cv2
import os
import csv
import pickle
import base64
import numpy as np
import face_recognition
from datetime import datetime
from IPython.display import display, Javascript
from google.colab.output import eval_js


In [7]:
def take_multiple_frames(n=5):
    display(Javascript(f"""
    async function captureMultiple() {{
        const div = document.createElement('div');
        const video = document.createElement('video');
        const button = document.createElement('button');
        const info = document.createElement('p');

        button.textContent = '📸 Capture Next';
        info.textContent = 'Click button to capture images';
        div.appendChild(info);
        div.appendChild(video);
        div.appendChild(button);
        document.body.appendChild(div);

        const stream = await navigator.mediaDevices.getUserMedia({{video: true}});
        video.srcObject = stream;
        await video.play();

        let images = [];

        for (let i = 0; i < {n}; i++) {{
            info.textContent = `Capture image ${{i+1}}/{n}`;
            await new Promise(resolve => button.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            images.push(canvas.toDataURL('image/jpeg', 0.8));
        }}

        stream.getTracks().forEach(track => track.stop());
        div.remove();
        return images;
    }}
    """))

    data = eval_js("captureMultiple()")

    frames = []
    for img in data:
        decoded = base64.b64decode(img.split(",")[1])
        frame = cv2.imdecode(np.frombuffer(decoded, np.uint8), cv2.IMREAD_COLOR)
        frames.append(frame)

    return frames


In [8]:
class FaceAttendanceSystemColab:
    def __init__(self):
        self.encodings_file = "face_encodings.pkl"
        self.attendance_file = "attendance_log.csv"
        self.known_faces = {}  # name -> list of encodings

        os.makedirs("registered_faces", exist_ok=True)
        self.load_encodings()

        if not os.path.exists(self.attendance_file):
            with open(self.attendance_file, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["Name", "Date", "Time", "Status"])

    def save_encodings(self):
        with open(self.encodings_file, "wb") as f:
            pickle.dump(self.known_faces, f)

    def load_encodings(self):
        if os.path.exists(self.encodings_file):
            with open(self.encodings_file, "rb") as f:
                self.known_faces = pickle.load(f)
            print(f"✅ Loaded {len(self.known_faces)} registered persons")
        else:
            self.known_faces = {}

    def register_face(self, name, samples=5):
        print(f"\n📸 Registering {name} ({samples} images)")

        if name not in self.known_faces:
            self.known_faces[name] = []

        for i in range(samples):
            print(f"➡️ Capture image {i+1}/{samples}")
            frame = take_frame()
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            locations = face_recognition.face_locations(rgb)
            if not locations:
                print("❌ No face detected, retry")
                continue

            encoding = face_recognition.face_encodings(rgb, locations)[0]
            self.known_faces[name].append(encoding)

            cv2.imwrite(f"registered_faces/{name}_{i+1}.jpg", frame)

        self.save_encodings()
        print(f"✅ {name} registered with {len(self.known_faces[name])} images")

    def mark_attendance(self, name):
        now = datetime.now()
        date = now.strftime("%Y-%m-%d")
        time = now.strftime("%H:%M:%S")

        with open(self.attendance_file, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, date, time, "Present"])

        print(f"✅ Attendance marked for {name}")

    def run_attendance(self):
        print("\n📷 Capture image for attendance")
        frame = take_frame()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        locations = face_recognition.face_locations(rgb)
        encodings = face_recognition.face_encodings(rgb, locations)

        if not encodings:
            print("❌ No face detected")
            return

        for face_encoding in encodings:
            best_name = "Unknown"
            best_distance = 1.0

            for name, enc_list in self.known_faces.items():
                distances = face_recognition.face_distance(enc_list, face_encoding)
                avg_distance = np.mean(distances)

                if avg_distance < best_distance:
                    best_distance = avg_distance
                    best_name = name

            if best_distance < 0.55:
                self.mark_attendance(best_name)
            else:
                print("❌ Unknown face detected")

    def view_attendance(self):
        print("\n📊 Attendance Log")
        with open(self.attendance_file) as f:
            for row in csv.reader(f):
                print(row)


In [9]:
system = FaceAttendanceSystemColab()


In [10]:
system.register_face("Ahmad", samples=5)
system.register_face("Danial", samples=5)
system.register_face("Mark", samples=5)



📸 Registering Ahmad (5 images)
➡️ Capture image 1/5


<IPython.core.display.Javascript object>

➡️ Capture image 2/5


<IPython.core.display.Javascript object>

➡️ Capture image 3/5


...

➡️ Capture image 4/5


<IPython.core.display.Javascript object>

➡️ Capture image 5/5


<IPython.core.display.Javascript object>

✅ Mark registered with 5 images


In [18]:
system.run_attendance()



📷 Capture image for attendance


<IPython.core.display.Javascript object>

✅ Attendance marked for Mark


In [19]:
system.view_attendance()



📊 Attendance Log
['Name', 'Date', 'Time', 'Status']
['Ahmad', '2026-01-05', '18:47:10', 'Present']
['Danial', '2026-01-05', '18:47:48', 'Present']
['Mark', '2026-01-05', '18:48:21', 'Present']
['Danial', '2026-01-05', '18:49:13', 'Present']
['Ahmad', '2026-01-05', '18:49:27', 'Present']
['Mark', '2026-01-05', '18:51:34', 'Present']
